# We need the following libraries to complete this project:

In [155]:
%matplotlib inline
from typing import List, Tuple
from matplotlib import pyplot as plt
from mpnn.data import make_tfrecord
from mpnn.data import make_training_tuple
from sklearn.model_selection import train_test_split
from rdkit import Chem
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
import numpy as np
import json
from functools import partial

# 1: Formatting Data
This step is very similar to data saving step performed at homework 2: Message passing networks. However, we need to make some modifications to include geometric information. We will also take advantage of the mpnn library created for homework 2.

## 1.1 Load data

In [3]:
data = pd.read_json('data/qm9.json.gz', lines=True)
print(f'Loaded {len(data)} training entries')

Loaded 25000 training entries


Convert the SMILES to RDKit molecules. 

Make sure to add the Hydrogens in

In [5]:
%%time
data['mol'] = data['smiles_0'].apply(Chem.MolFromSmiles).apply(Chem.AddHs)

Wall time: 2.15 s


## 1.2 Create Lookup Table
Here, the function make_type_lookup_tables taken directly from 0_save-training-data.ipynb on the class github page and no change is required

In [6]:
def make_type_lookup_tables(mols: List[Chem.Mol]) -> Tuple[List[int], List[str]]:
    """Create lists of observed atom and bond types

    Args:
        mols: List of molecules used for our training set
    Returns:
        - List of atom types (elements)
        - List of bond types (elements)
    """

    # Initialize the lists
    atom_types = set()
    bond_types = set()

    # Get all types observed in these graphs
    for mol in mols:
        atom_types.update([x.GetAtomicNum() for x in mol.GetAtoms()])
        bond_types.update([x.GetBondType() for x in mol.GetBonds()])

    # Return as sorted lists
    return sorted(atom_types), sorted(bond_types)

In [7]:
atom_types, bond_types = make_type_lookup_tables(data['mol'])
print(f'Found {len(atom_types)} types of atoms: {atom_types}')
print(f'Found {len(bond_types)} types of bonds: {bond_types}')

Found 5 types of atoms: [1, 6, 7, 8, 9]
Found 4 types of bonds: [rdkit.Chem.rdchem.BondType.SINGLE, rdkit.Chem.rdchem.BondType.DOUBLE, rdkit.Chem.rdchem.BondType.TRIPLE, rdkit.Chem.rdchem.BondType.AROMATIC]


## 1.3 Get Geometry Information
We must add this additional function for the project. It takes some xyz file stored in given path and return the atomic coordinates as N-by-3 numpy arrays.

In [60]:
def get_XYZ(path, filename):
    with open(path + "/" + filename) as f:
        lines = f.readlines()
    f.close()
    atom_count = int(lines[0])
    endline = 2 + atom_count
    ret_val = np.zeros([atom_count, 3], dtype=float)
    for i in np.arange(2,endline):
        line = lines[i].replace("*^", "e")
        parsed_line = line.split()
        x = float(parsed_line[1])
        y = float(parsed_line[2])
        z = float(parsed_line[3])
        ret_val[i-2, :] = [x,y,z]
    return ret_val
    

# 1.4 Integrate Geometric Information into Existing Dataset
We must make some changes to convert_mol_to_dict function, originally shown in 0_save-training-data.ipynb to save geometric information alongside molecular type information.

In [41]:
# Note that a new input "xyz" (of type np.array) is added

def convert_mol_to_dict(mol: Chem.Mol, atom_types: List[int], bond_types: List[str], XYZ) -> dict:
    # Get the atom types, look them up in the atom_type list
    atom_type = [a.GetAtomicNum() for a in mol.GetAtoms()]
    atom_type_id = list(map(atom_types.index, atom_type))
    # Get the bond types and which atoms these connect
    connectivity = []
    edge_type = []
    for bond in mol.GetBonds():
        # Get information about the bond
        a = bond.GetBeginAtomIdx()
        b = bond.GetEndAtomIdx()
        b_type = bond.GetBondType()        
        # Store how they are connected
        connectivity.append([a, b])
        connectivity.append([b, a])
        edge_type.append(b_type)
        edge_type.append(b_type)
    edge_type_id = list(map(bond_types.index, edge_type))
    # Sort connectivity array by the first column
    #  This is needed for the MPNN code to efficiently group messages for
    #  each atom when performing the message passing step
    connectivity = np.array(connectivity)
    if connectivity.size > 0:
        # Skip a special case of a molecule w/o bonds
        inds = np.lexsort((connectivity[:, 1], connectivity[:, 0]))
        connectivity = connectivity[inds, :]

        # Tensorflow's "segment_sum" will cause problems if the last atom
        #  is not bonded because it returns an array
        if connectivity.max() != len(atom_type) - 1:
            smiles = convert_nx_to_smiles(graph)
            raise ValueError(f"Problem with unconnected atoms for {smiles}")
    else:
        connectivity = np.zeros((0, 2))

    return {
        'n_atom': len(atom_type),
        'n_bond': len(edge_type),
        'atom': atom_type_id,
        'bond': edge_type_id,
        'connectivity': connectivity,
        "XYZ": XYZ
    }

We use the following loop to create a create a dictionary for each molecule in the dataset and attach it to the dataset

In [99]:
dicts = []
XYZ_path = "data/XYZ"
for ind in data.index:
    filename = data.loc[ind, "filename"]
    XYZ = get_XYZ(XYZ_path, filename)
    new_data = convert_mol_to_dict(data.loc[ind, "mol"], atom_types, bond_types, XYZ)
    dicts.append(new_data)
data["dict"] = dicts

## 1.6 Save the Data as TFRecords


In [64]:
train_data, test_data = train_test_split(data, shuffle=True, train_size=0.9)

In [65]:
train_data, valid_data = train_test_split(train_data, train_size=0.9)

Save the data in TFDataset format in "protobuf" files. Note that the y-variables we save are: zero-point energy, dipole moment, homo and lumo.

In [72]:
for name, dataset in zip(['train', 'valid', 'test'], [train_data, valid_data, test_data]):
    with tf.io.TFRecordWriter(f'data/{name}_data.proto') as writer:
        for _, entry in tqdm(dataset.iterrows(), desc=name):
            record = entry['dict']
            for o in ['zpe', "mu", "homo", "lumo"]:
                record[o] = entry[o]
            writer.write(make_tfrecord(record))

train: 20250it [00:43, 461.31it/s]
valid: 2250it [00:04, 450.84it/s]
test: 2500it [00:05, 432.99it/s]


# 2: Creating Data loader
We create a data loader similar to that created in Homework2, but modified to include more flexibility as to whether to include geometric information or not.

## 2.1 Helper functions
These following three functions are similar to those in 1_explain-data-loader.ipynb on class github page. They are modified to handle more y-variables as well as geometric info. The first function, parse_records, either picks up XYZ information from .proto files or not, depending on user specification of "include_XYZ" variable

In [146]:
def parse_records(example_proto, include_XYZ, target_name, target_shape: Sequence[int] = ()):
    default_target = np.zeros(target_shape) * np.nan
    if include_XYZ:
        features = {
            target_name: tf.io.FixedLenFeature(target_shape, tf.float32, default_value=default_target),
            'n_atom': tf.io.FixedLenFeature([], tf.int64),
            'n_bond': tf.io.FixedLenFeature([], tf.int64),
            'connectivity': tf.io.VarLenFeature(tf.int64),
            'atom': tf.io.VarLenFeature(tf.int64),
            'bond': tf.io.VarLenFeature(tf.int64),
            'XYZ': tf.io.VarLenFeature(tf.float32)
        }
    else:
        features = {
            target_name: tf.io.FixedLenFeature(target_shape, tf.float32, default_value=default_target),
            'n_atom': tf.io.FixedLenFeature([], tf.int64),
            'n_bond': tf.io.FixedLenFeature([], tf.int64),
            'connectivity': tf.io.VarLenFeature(tf.int64),
            'atom': tf.io.VarLenFeature(tf.int64),
            'bond': tf.io.VarLenFeature(tf.int64)
        }
    return tf.io.parse_example(example_proto, features)

We inclue the original functions in 1_explain-data-loader.ipynb to handle input without geometry:

In [147]:
def prepare_for_batching(dataset):
    for c in ['atom', 'bond', 'connectivity']:
        expanded = tf.expand_dims(dataset[c].values, axis=0, name=f'expand_{c}')
        dataset[c] = tf.RaggedTensor.from_tensor(expanded)
    return dataset

In [148]:
def combine_graphs(batch):
    for c in ['atom', 'bond', 'connectivity']:
        expanded = tf.expand_dims(batch[c].values, axis=0, name=f'expand_{c}')
        batch[c] = tf.RaggedTensor.from_tensor(expanded).flat_values
    batch_size = tf.size(batch['n_atom'], name='batch_size')
    mol_id = tf.range(batch_size, name='mol_inds')
    batch['node_graph_indices'] = tf.repeat(mol_id, batch['n_atom'], axis=0)
    batch['bond_graph_indices'] = tf.repeat(mol_id, batch['n_bond'], axis=0)
    batch['connectivity'] = tf.reshape(batch['connectivity'], (-1, 2))
    offset_values = tf.cumsum(batch['n_atom'], exclusive=True)
    offsets = tf.repeat(offset_values, batch['n_bond'], name='offsets', axis=0)
    batch['connectivity'] += tf.expand_dims(offsets, 1)
    return batch

And we create their dopplegangers that do handle geometry.

In [149]:
def prepare_for_batching_XYZ(dataset):
    for c in ['atom', 'bond', 'connectivity', 'XYZ']:
        expanded = tf.expand_dims(dataset[c].values, axis=0, name=f'expand_{c}')
        dataset[c] = tf.RaggedTensor.from_tensor(expanded)
    return dataset

In [150]:
def combine_graphs_XYZ(batch):
    for c in ['atom', 'bond', 'connectivity',"XYZ"]:
        expanded = tf.expand_dims(batch[c].values, axis=0, name=f'expand_{c}')
        batch[c] = tf.RaggedTensor.from_tensor(expanded).flat_values
    batch_size = tf.size(batch['n_atom'], name='batch_size')
    mol_id = tf.range(batch_size, name='mol_inds')
    batch['node_graph_indices'] = tf.repeat(mol_id, batch['n_atom'], axis=0)
    batch['bond_graph_indices'] = tf.repeat(mol_id, batch['n_bond'], axis=0)
    batch['connectivity'] = tf.reshape(batch['connectivity'], (-1, 2))
    # Note that geometric information needs to be reshaped to N-by-3 the just as connectivity information
    batch['XYZ'] = tf.reshape(batch['XYZ'], (-1, 3))
    offset_values = tf.cumsum(batch['n_atom'], exclusive=True)
    offsets = tf.repeat(offset_values, batch['n_bond'], name='offsets', axis=0)
    batch['connectivity'] += tf.expand_dims(offsets, 1)
    return batch

## 2.2 Put Everything Together for a single Data Loader
This function is modified from make_data_loader in data.py in mpnn library. It includes an new parameter "include_XYZ"

In [163]:
def make_data_loader(file_path, 
                     include_XYZ: bool,
                     batch_size=32, shuffle_buffer=None,
                     n_threads=tf.data.experimental.AUTOTUNE, shard=None,
                     cache: bool = False, output_property: str="zpe",
                     output_shape: Sequence[int] = ()) -> tf.data.TFRecordDataset:
    """
    Args:
        file_path (str): Path to the training set
        include_XYZ (bool): whether data will include atomic coordinates
        batch_size (int): Number of graphs per training batch
        shuffle_buffer (int): Width of window to use when shuffling training entries
        n_threads (int): Number of threads over which to parallelize data loading
        cache (bool): Whether to load the whole dataset into memory
        shard ((int, int)): Parameters used to shared the dataset: (size, rank)
        output_property (str): Which property to use as the output
        output_shape ([int]): Shape of the output property
    Returns:
        (tf.data.TFRecordDataset) An infinite dataset generator
    """
    r = tf.data.TFRecordDataset(file_path)
    if cache:
        r = r.cache()
    if shuffle_buffer is not None:
        r = r.shuffle(shuffle_buffer)
    if shard is not None:
        r = r.shard(*shard)
    parse = partial(parse_records, include_XYZ=include_XYZ, target_name=output_property, target_shape=output_shape)
    if not(include_XYZ):
        r = r.batch(batch_size).map(parse, n_threads).map(prepare_for_batching, n_threads)
        r = r.map(combine_graphs, n_threads)
    else:
        r = r.batch(batch_size).map(parse, n_threads).map(prepare_for_batching_XYZ, n_threads)
        r = r.map(combine_graphs_XYZ, n_threads)
    train_tuple = partial(make_training_tuple, target_name=output_property)
    return r.map(train_tuple)

## 2.3 Checking Data Loader

In [178]:
data_file="data/train_data.proto"

In [179]:
data = make_data_loader(data_file, include_XYZ=True, batch_size=2, output_property="zpe")
#next(iter(data))

In [180]:
data = make_data_loader(data_file, include_XYZ=False, batch_size=2, output_property="zpe")
#next(iter(data))

The data loader is confirmed to be capable of making two input sets identical except for the inclusion of XYZ data. The actually checking step is commented out to occupy less space. 